# Objective

<p>
    <span style='font-family:Arial'>
    Classify spam text message
    </span>
</p> 

In [1]:
import numpy as np 
import pandas as pd 
import nltk as nlp
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [2]:
# load data
data = pd.read_csv("../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")

In [3]:
data.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# convert spam to 1, not spam to 0
data["Category"] = [1 if category == "spam" else 0 for category in data["Category"]]

In [5]:
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   
    description = nlp.word_tokenize(description)
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description) 

In [7]:
max_features = 3000 
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("The most used {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

The most used 3000 words: ['aah', 'aathi', 'abi', 'ability', 'abiola', 'abj', 'able', 'absolutly', 'abt', 'abta', 'aburo', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'acl', 'aco', 'acted', 'acting', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'administrator', 'admirer', 'admit', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'aeronautics', 'aeroplane', 'affair', 'affection', 'afraid', 'aft', 'afternoon', 'aftr', 'ag', 'agalla', 'age', 'agent', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahmad', 'aid', 'aight', 'ain', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'ali', 'alive', 'allah', 'allow', 'allowed', 'alright', 'alrite', 'alwys', 'amazing', 'american', 'amp', 'amt',

In [8]:
y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [9]:
nb = GaussianNB()
nb.fit(x_train,y_train)
print("Gaussian NB model Accuracy: {}".format(nb.score(x_test,y_test)))

Gaussian NB model Accuracy: 0.8763440860215054


In [10]:
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("Logistic Regression model Accuracy: {}".format(lr.score(x_test,y_test)))

Logistic Regression model Accuracy: 0.974910394265233


In [11]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
print('KNN (K=3) model accuracy is: ',knn.score(x_test,y_test))

KNN (K=3) model accuracy is:  0.9408602150537635


In [12]:
x_train = x_train.reshape(5014,3000,1)
x_test = x_test.reshape(558,3000,1)

In [13]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

regressor.fit(x_test, y_test, epochs = 3, batch_size = 32)


Epoch 1/3
18/18 [==============================] - 15s 392ms/step - loss: 0.1205 - accuracy: 0.8697
Epoch 2/3
18/18 [==============================] - 7s 391ms/step - loss: 0.1238 - accuracy: 0.8582
Epoch 3/3
18/18 [==============================] - 7s 390ms/step - loss: 0.1197 - accuracy: 0.8634
